In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(1)
sns.set(style="darkgrid")

import warnings
warnings.filterwarnings('ignore')

<h3>Кармазин Василий ПИН-43</h3>
<br><br>
<center>
    <h1>Лабораторная работа №4</h1>
    <h2>Модель "Хищник-Жертва"</h2>
</center>

<h3>Задание:</h3>
Построить модель Мальтуса взаимоотношения "хищник-жертва".

Объектом исследования является модель двувидовой борьбы (соперничества), а именно модель взаимодействия двух популяций: хищников и их жертв. Взаимодействие жертвы с хищником выражается в изменении численности жертвы, которая в свою очередь сказывается на численности хищника.

<center>
    <h3>1. Построение модели</h3>
</center>

<h4>Описание модели:</h4>

В данной модели мы предполагаем, что: 
- Численность популяции зависит только от времени, естественная рождаемость и смертность не учитывается
- Жертвы питаются от среды, а хищники питаются жертвами
- Скорость роста численности жертвы уменьшается пропорционально численности хищников
- Эффект насыщения численности обоих популяций не учитывается

<h4>Пусть:</h4>

- $V(t)$ - численность популяции жертв
- $b_v$ - коэффициент рождаемости жертв
- $d_v$ - коэффициент смертности жертв


- $P(t)$ - численность популяции хищников
- $b_p$ - коэффициент рождаемости хищников
- $d_p$ - коэффициент смертности хищников

Модель Мальтуса имеет вид:
<center>
    $\dfrac{dN}{dt} = (b(N,t) - d(N,t))N$
</center>

Решение уравнения
<center>
    $N(t) = N_0 * e^{\epsilon t}, N_0 = N(0)$
</center>


Считаем, что все коэффициенты взаимодействия между видами положительные
Используя Модель Мальтуса с необходимыми поправками, учитывающими поедание жертв хищниками, приходим к следующей системе, носящей имя модели Вольтерры — Лотки:

<center>
    $\begin{cases} 
        \dfrac{dV}{dt} = (b_v - d_vP)V \\ 
        \dfrac{dP}{dt} = (- b_p + d_pV)P 
    \end{cases}$
</center>

С начальными значениями $V(0) = V_0, P(0) = P_0$


Исключая $dt$ из системы получим

<center>
$\dfrac{(b_v - d_vP)V}{(- b_p + d_pV)P} = \dfrac{dV}{dP}$
</center>

Упростим

<center>
$(b_v - d_vP)V dP = (- b_p + d_pV)P dV$
</center>

Разделим на $VP$

<center>
$b_v\dfrac{dP}{P} - d_vdP = - b_p\dfrac{dV}{V} + d_pdV$
</center>

Проинтегрировав дифференциальное уравнение получим

<center>
$b_v\int_{}{}\dfrac{dP}{P} - d_v\int_{}{}dP = -b_p\int_{}{}\dfrac{dV}{V} + d_p\int_{}{}dV$
</center>

<center>
$b_v \ln{P} - d_vP = -b_p \ln{V} + d_pV + Const$, где $Const = V(0)+P(0)$
</center>

Экспоненциируем

<center>
$P^{b_v}e^{-d_vP} = V^{-b_p}e^{d_pV} * Const$
</center>

In [2]:
def pend(y_0, t, b_v, d_v, b_p, d_p):
    V, P = y_0
    return (
        (b_v - d_v*P) * V,
        (-b_p + d_p*V) * P,
    ) 

In [3]:
# Зададим конфигурацию системы
V_0, P_0 = 10, 10

b_v, d_v = 1, 1
b_p, d_p = 1, 1

<center>
    <h3>2. Визуализация</h3>
</center>

In [13]:
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, Slider
from bokeh.plotting import figure
from bokeh.io import show, output_notebook

from scipy.integrate import odeint

output_notebook()

Loading BokehJS ...

In [21]:
t = np.linspace(0, 200, 1000)
def bkapp(doc):
    sol = odeint(pend, [V_0, P_0], t, args=(b_v, d_v, b_p, d_p))
    data = {'t': t, 'x': sol[:,0], 'y': sol[:,1]}
    source = ColumnDataSource(data=data)

    plot = figure(x_axis_label='time', y_range=(0, 30), plot_height=400,
                  y_axis_label='population',
                  title='Модель "Хищник-Жертва"')
    plot.line('t', 'x', source=source, line_color='green')
    plot.line('t', 'y', source=source, line_color='red')
    
    
    plot_vp = figure(x_axis_label='Популяция Жертв',plot_height=400,
                  y_axis_label='Популяция Хищников')
    plot_vp.line('x', 'y', source=source, line_color='blue')
    
    def callback(param):
        def _callback(attr, old, new):
            global P_0, V_0, b_v, d_v, b_p, d_p
            if param == 'V_0': V_0 = new
            if param == 'P_0': P_0 = new
            if param == 'b_v': b_v = new
            if param == 'd_v': d_v = new
            if param == 'b_p': b_p = new
            if param == 'd_p': d_p = new
                
            sol = odeint(pend, [V_0, P_0], t, args=(b_v, d_v, b_p, d_p))
            data['x'], data['y'] = sol[:,0], sol[:,1]
            source.data = ColumnDataSource(data=data).data
        return _callback
        
    sliders = {
        'V_0': Slider(start=0, end=100, value=V_0, step=1, title="Начальная популяция жертв"),
        'b_v': Slider(start=0, end=5, value=b_v, step=0.05, title="Коэффициент рождаемости жертв"),
        'd_v': Slider(start=0, end=5, value=d_v, step=0.05, title="Коэффициент смертности жертв"),
        
        'P_0': Slider(start=0, end=100, value=P_0, step=1, title="Начальная популяция хищников"),        
        'b_p': Slider(start=0, end=5, value=b_p, step=0.05, title="Коэффициент рождаемости хищников"),
        'd_p': Slider(start=0, end=5, value=d_p, step=0.05, title="Коэффициент смертности хищников"),
    }
    for param, slider in sliders.items():
        slider.on_change('value', callback(param))

    layout = row(
        column(plot, plot_vp),
        column(*sliders.values()),
    )
    doc.add_root(layout)

In [22]:
show(bkapp)

Отклонение от этого состояния приводит к колебаниям численности жертв и хищников, аналогичным колебаниям гармонического осциллятора. Как и в случае гармонического осциллятора, это поведение не является структурно устойчивым: малое изменение модели может привести к качественному изменению поведения. 

<h4>Улучшение модели:</h4>

Добавим эффект насыщения хищников от времени $h$, тогда уравнение примет вид:

<center>
    $\dfrac{dV}{dt} = (b_v - d_vP - hV)V$
<center>

In [25]:
def pend(y_0, t, b_v, d_v, b_p, d_p, h):
    V, P = y_0
    return (
        (b_v - d_v*P - h*V) * V,
        (-b_p + d_p*V) * P,
    ) 

In [26]:
# Зададим конфигурацию системы
V_0, P_0 = 10, 10

b_v, d_v = 1, 1
b_p, d_p = 1, 1
h = 0.2

In [33]:
t = np.linspace(0, 200, 1000)
def bkapp(doc):
    sol = odeint(pend, [V_0, P_0], t, args=(b_v, d_v, b_p, d_p, h))
    data = {'t': t, 'x': sol[:,0], 'y': sol[:,1]}
    source = ColumnDataSource(data=data)

    plot = figure(x_axis_label='time', y_range=(0, 30), plot_height=400,
                  y_axis_label='population',
                  title='Модель "Хищник-Жертва", с насыщением')
    plot.line('t', 'x', source=source, line_color='green')
    plot.line('t', 'y', source=source, line_color='red')
    
    
    plot_vp = figure(x_axis_label='Популяция Жертв',plot_height=400,
                  y_axis_label='Популяция Хищников')
    plot_vp.line('x', 'y', source=source, line_color='blue')
    
    def callback(param):
        def _callback(attr, old, new):
            global P_0, V_0, b_v, d_v, b_p, d_p, h
            if param == 'V_0': V_0 = new
            if param == 'P_0': P_0 = new
            if param == 'b_v': b_v = new
            if param == 'd_v': d_v = new
            if param == 'b_p': b_p = new
            if param == 'd_p': d_p = new
            if param == 'h': h = new
                
            sol = odeint(pend, [V_0, P_0], t, args=(b_v, d_v, b_p, d_p, h))
            data['x'], data['y'] = sol[:,0], sol[:,1]
            source.data = ColumnDataSource(data=data).data
        return _callback
        
    sliders = {
        'V_0': Slider(start=0, end=100, value=V_0, step=1, title="Начальная популяция жертв"),
        'b_v': Slider(start=0, end=5, value=b_v, step=0.05, title="Коэффициент рождаемости жертв"),
        'd_v': Slider(start=0, end=5, value=d_v, step=0.05, title="Коэффициент смертности жертв"),
        
        'P_0': Slider(start=0, end=100, value=P_0, step=1, title="Начальная популяция хищников"),        
        'b_p': Slider(start=0, end=5, value=b_p, step=0.05, title="Коэффициент рождаемости хищников"),
        'd_p': Slider(start=0, end=5, value=d_p, step=0.05, title="Коэффициент смертности хищников"),
        'h': Slider(start=0, end=2, value=h, step=0.005, title="Коэффициент насыщенности хищников"),
    }
    for param, slider in sliders.items():
        slider.on_change('value', callback(param))

    layout = row(
        column(plot, plot_vp),
        column(*sliders.values()),
    )
    doc.add_root(layout)

In [34]:
show(bkapp)

При учете коэффициента насыщения зависимость численность будет стремиться к точке равновесия.